**Task 1:** Image classification using a MLFFNN with two hidden layers, and with Deep CNN 
features for an image as the input to the MLFFNN, with (a) VGGNet as Deep CNN and (b) 
GoogLeNet as Deep CNN.